### Vorbereitung der Programmierumgebung

In [1]:
# Import benötigter Pakete
import arcpy
from arcpy import env
from arcpy.sa import *
import math
import os
import pandas as pd

In [2]:
# Umgebungseinstellungen
arcpy.env.workspace = r"Beispieldaten"  # Pfad zum Arbeitsverzeichnis
arcpy.env.overwriteOutput = True
os.chdir(arcpy.env.workspace)


### Daten einladen

In [5]:
# Referenz-Raster (Sichtfeld)
viewshed = "/View15_30q3_binär.tif"

# Verzeichnis mit den Eingabe-Rastern (Parameter)
parameter_folder = r"/Parameter"
raster_list = [f for f in os.listdir(parameter_folder) if f.endswith(".tif")]

# Landslide Raster (Binär: 1 = Landslide, 0 = Kein Landslide)
landslide_raster = "/sli_train_80_binary.tif"

# Ausgabeordner
output_folder = os.path.join(arcpy.env.workspace, "/Statistical_Index")


### Berechnung des Statistical Index für jeden Parameter der Liste 

In [6]:
for raster in raster_list:
    si_values = {}  # Dictionary zur Speicherung der SI-Werte
    input_raster = os.path.join(arcpy.env.workspace, "Parameter", raster)

    print(f"Verarbeitung von: {raster}")
    # Raster auf Sichtfeld zuschneiden
    clipped_raster = ExtractByMask(input_raster, viewshed)
    print(f"{raster} wurde erfolgreich zugeschnitten")

    # Zonal Statistics Table (Anzahl Hangrutschung pro Parameterklasse)
    tabulate_output = os.path.join(output_folder, f"Tabulate_{raster.replace('.tif', '')}.dbf")
    zone_field = "Value"
    arcpy.sa.ZonalStatisticsAsTable(clipped_raster, zone_field, landslide_raster, tabulate_output, "DATA", "SUM")
    print(f"Tabulate-Ausgabe gespeichert unter: {tabulate_output}")

    # Gesamte Anzahl der Landrutsche berechnen
    total_landslides = 0
    with arcpy.da.SearchCursor(tabulate_output, ['Value', 'COUNT']) as cursor:
        for row in cursor:
            total_landslides += row[1]
    print(f"Gesamtanzahl der Landrutsche: {total_landslides}")

    # Pixelanzahl pro Klasse berechnen
    total_pixel = os.path.join(output_folder, f"total_pixel_{raster.replace('.tif', '')}.dbf")
    arcpy.sa.ZonalStatisticsAsTable(clipped_raster, "Value", clipped_raster, total_pixel, "DATA", "SUM")
    
    # Gesamtzahl gültiger Pixel im Sichtfeld
    total_pixel_count = 0
    with arcpy.da.SearchCursor(total_pixel, ["COUNT"]) as cursor:
        for row in cursor:
            total_pixel_count += row[0]
    print(f"Anzahl gültiger Pixel im Parameter-Raster: {total_pixel_count}")

    # Pixel pro Klasse in Dictionary speichern
    pixel_counts_per_class = {}
    with arcpy.da.SearchCursor(total_pixel, ["Value", "COUNT"]) as cursor:
        for row in cursor:
            class_value = row[0]
            count = row[1]
            pixel_counts_per_class[class_value] = count

    # SI-Werte pro Klasse berechnen
    with arcpy.da.SearchCursor(tabulate_output, ["Value", "COUNT"]) as cursor:
        for row in cursor:
            class_value = row[0]
            landslide_count = row[1]
            class_pixel_count = pixel_counts_per_class.get(class_value, 0)

            if class_pixel_count > 0 and total_pixel_count > 0 and total_landslides > 0:
                si = math.log((landslide_count / class_pixel_count) / (total_landslides / total_pixel_count))
                si_values[class_value] = si
            else:
                si_values[class_value] = 0

    # Kopiere das ursprüngliche Raster und schreibe SI-Werte hinein
    full_output_raster = os.path.join(output_folder, raster.replace(".tif", "_SI.tif"))
    arcpy.management.CopyRaster(input_raster, full_output_raster)

    # Attributtabelle erstellen und SI-Feld hinzufügen
    arcpy.management.BuildRasterAttributeTable(full_output_raster, "OVERWRITE")
    arcpy.management.AddField(full_output_raster, "SI", "DOUBLE")

    # SI-Werte ins neue Feld schreiben
    with arcpy.da.UpdateCursor(full_output_raster, ["Value", "SI"]) as cursor:
        for row in cursor:
            class_value = row[0]
            si = si_values.get(class_value, 0)
            row[1] = si
            cursor.updateRow(row)

    print(f"SI-Werte in {full_output_raster} geschrieben")
    print("###########################################################################")
                               
print("Prozess abgeschlossen")

Verarbeitung von: aspect_reclass_reproject.tif
aspect_reclass_reproject.tif wurde erfolgreich zugeschnitten
Tabulate-Ausgabe gespeichert unter: C:\Users\Marlenelokal\OneDrive\Dokumente\UniJena\Geo403_Geoinfo_Projekt\Geoinfo_Projekt\Automatisation\data\Statistical_Index\Tabulate_aspect_reclass_reproject.dbf
Gesamtanzahl der Landrutsche: 1432.0
Anzahl gültiger Pixel im Parameter-Raster: 1540692.0
SI-Werte in C:\Users\Marlenelokal\OneDrive\Dokumente\UniJena\Geo403_Geoinfo_Projekt\Geoinfo_Projekt\Automatisation\data\Statistical_Index\aspect_reclass_reproject_SI.tif geschrieben
###########################################################################
Verarbeitung von: curvature_planar_reclass_reproject.tif
curvature_planar_reclass_reproject.tif wurde erfolgreich zugeschnitten
Tabulate-Ausgabe gespeichert unter: C:\Users\Marlenelokal\OneDrive\Dokumente\UniJena\Geo403_Geoinfo_Projekt\Geoinfo_Projekt\Automatisation\data\Statistical_Index\Tabulate_curvature_planar_reclass_reproject.dbf
Gesamt

### Gesamtgefährdung

In [7]:
# SI aufsummieren, Gesamtgefährdung erstellen

# Verzeichnis mit den Eingabe-Rastern
SI_raster_list = [f for f in os.listdir(output_folder) if f.endswith(".tif")]
si_raster_paths = [os.path.join(output_folder, r) for r in SI_raster_list]

# Lookup-Raster erstellen (temporär), da die Spalte SI nicht direkt addiert werden kann
lookup_rasters = [Lookup(raster, "SI") for raster in si_raster_paths]
# Summe berechnen
si_sum = CellStatistics(lookup_rasters, "SUM", "DATA")
si_sum.save(os.path.join(output_folder, "SI_sum.tif")) # Layer speichern
print("Gefährdungslayer wurde erfolgreich erstellt.") # Ausgabe

Gefährdungslayer wurde erfolgreich erstellt.


### SI Standardisierung

In [9]:
output = "si_sum_standardized.tif"
# Min und Max Werte des si_sum Layers berechnen
min_value = arcpy.management.GetRasterProperties(si_sum, "MINIMUM")
max_value = arcpy.management.GetRasterProperties(si_sum, "MAXIMUM")

# Min und Max als Variablen extrahieren
min_value = float(min_value.getOutput(0))
max_value = float(max_value.getOutput(0))

# Raster standardisieren: (Wert - Min) / (Max - Min)
standardized_raster = (Raster(input_raster) - min_value) / (max_value - min_value)

# Ausgabe-Raster speichern
standardized_raster.save(output)
